1. IMPORTING


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras import layers, models
import numpy as np
import os
import matplotlib.pyplot as plt
